In [1]:
import sys
from pathlib import Path

# This points to /desktop/open-books/
project_root = str(Path.cwd().parent.parent)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now 'from src.ingestion...' will work because 'src' is a folder inside project_root


In [2]:
from abc import ABC, abstractmethod
from src.utils.config import LLMConfig
from openai import OpenAI
import ollama  # pip install ollama

In [3]:
from pydantic import BaseModel, Field
from typing import Literal, Optional
from abc import ABC, abstractmethod


class BaseGenerator(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        """Takes a formatted prompt and returns the LLM's text response."""
        pass


class OpenAIGenerator(BaseGenerator):
    def __init__(self, config: LLMConfig):
        self.client = OpenAI(api_key=config.api_key)
        self.model = config.model_name
        self.temperature = config.temperature

    def generate(self, prompt: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
        )
        return response.choices[0].message.content



""" class OllamaGenerator(BaseGenerator):
    def __init__(self, config: LLMConfig):
        self.model = config.model_name
        self.temperature = config.temperature
        self.host = config.base_url or "http://localhost:11434"
    
    def generate(self, prompt: str) -> str:
        response = ollama.chat(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            options={
                "temperature": self.temperature,
            }
        )
        return response['message']['content'] """

' class OllamaGenerator(BaseGenerator):\n    def __init__(self, config: LLMConfig):\n        self.model = config.model_name\n        self.temperature = config.temperature\n        self.host = config.base_url or "http://localhost:11434"\n\n    def generate(self, prompt: str) -> str:\n        response = ollama.chat(\n            model=self.model,\n            messages=[{"role": "user", "content": prompt}],\n            options={\n                "temperature": self.temperature,\n            }\n        )\n        return response[\'message\'][\'content\'] '

In [4]:
from src.utils.config import LLMConfig

class OllamaManager:
    @staticmethod
    def ensure_ready(model_name: str = "llama3.2"):
        """One-liner to ensure Ollama + model are ready"""
        try:
            # Try to list models (checks if server is running)
            models = ollama.list()
            
            # Check if model exists
            if not any(model_name in m['name'] for m in models['models']):
                print(f"📥 Downloading {model_name}...")
                ollama.pull(model_name)
            
            print(f"✅ Ready to use {model_name}")
            return True
            
        except Exception as e:
            print(f"⚠️ Ollama not running. Please start it:")
            print(f"   Run in terminal: ollama serve")
            return False

# Usage - just one line!
OllamaManager.ensure_ready("llama3.2")
class OllamaGenerator(BaseGenerator):
    def __init__(self, config: LLMConfig, auto_setup: bool = True):
        self.model = config.model_name
        self.temperature = config.temperature
        
        # Auto-check if requested
        if auto_setup:
            OllamaManager.ensure_ready(self.model)
    
    def generate(self, prompt: str) -> str:
        try:
            response = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={"temperature": self.temperature}
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}. Is Ollama running? Run 'ollama serve'"

⚠️ Ollama not running. Please start it:
   Run in terminal: ollama serve


In [5]:
class QueryConstructor(ABC):
    @abstractmethod
    def refine_query(self, query: str) -> list[str]:
        pass

In [6]:
class MultiQueryConstructor(QueryConstructor):
    def __init__(self,generator: BaseGenerator) -> None:
        super().__init__()
        self.generator = generator
        self.template = """You are an AI language model assistant. Your task is to generate five 
        different versions of the given user question to retrieve relevant documents from a vector 
        database. By generating multiple perspectives on the user question, your goal is to help
        the user overcome some of the limitations of the distance-based similarity search. 
        Provide these alternative questions separated by newlines. Original question: {question}"""

    def refine_query(self, query: str) -> list[str]:
        prompt = self.template.format(question=query)
        response = self.generator.generate(prompt)
        
        queries = [q.strip() for q in response.split('\n') if q.strip()]
        
        return [query] + queries

In [7]:
from src.shared.models import SearchResult
from typing import List

In [8]:
class BaseQueryAnswerer(ABC):
    @abstractmethod
    def answer(self,result_search:List[SearchResult],query:str)->str:
        pass

In [9]:
from typing import List


from shared.models import SearchResult

In [10]:
class QueryAnswerer(BaseQueryAnswerer):
    def __init__(self, generator: BaseGenerator) -> None:
        super().__init__()
        self.generator = generator
        self.template = """You are a helpful assistant answering questions based on provided context.

Context:
{context}

Question: {question}

Instructions:
- Answer based only on the provided context
- If the context doesn't contain enough information, say so
- Be concise but complete
- Cite relevant parts of the context when possible

Answer:"""
    
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        if not result_search:
            return "No relevant documents found to answer this question."
        
        context_parts = []
        for i, result in enumerate(result_search, 1):
            context_parts.append(f"[Document {i}]\n{result.content}")
        
        context = "\n\n".join(context_parts)
        
        prompt = self.template.format(context=context, question=query)
        answer = self.generator.generate(prompt)
        
        return answer.strip()

In [11]:
class QueryAnswererWithSources(BaseQueryAnswerer):
    def __init__(self, generator: BaseGenerator) -> None:
        super().__init__()
        self.generator = generator
        self.template = """Answer the question based on the context below. When using information from a document, reference it by its number [1], [2], etc.

Context:
{context}

Question: {question}

Provide your answer with citations:"""
    
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        if not result_search:
            return "No relevant documents found."
        
        context_parts = []
        for i, result in enumerate(result_search, 1):
            metadata = f"Source: {result.metadata.source_doc_title}" if hasattr(result, 'metadata') else ""
            context_parts.append(f"[{i}] {result.content}\n{metadata}")
        
        context = "\n\n".join(context_parts)
        prompt = self.template.format(context=context, question=query)
        
        return self.generator.generate(prompt).strip()

In [12]:
from src.ingestion.vector_store.stores import ChromaStore

/home/moad/desktop/open-books/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
class RAGPipeline:
    def __init__(
        self,
        query_constructor: QueryConstructor,
        vector_store :ChromaStore, 
        answerer: BaseQueryAnswerer
    ):
        self.query_constructor = query_constructor
        self.vector_store = vector_store
        self.answerer = answerer
    
    def query(self, user_query: str, top_k: int = 5) -> str:
        enhanced_queries = self.query_constructor.refine_query(user_query)
        
        all_results = []
        results = self.vector_store.query(enhanced_queries,n_result=top_k)
        all_results.extend(results)
        
        top_results = all_results[:top_k]
        
        answer = self.answerer.answer(top_results, user_query)
        
        return answer

In [14]:
# Just run this - it handles everything!
from src.utils.config import settings



# This auto-checks Ollama and downloads model if needed
generator = OllamaGenerator(settings.llm, auto_setup=True)

# Use it!
response = generator.generate("Explain RAG in simple terms")
print(response)

⚠️ Ollama not running. Please start it:
   Run in terminal: ollama serve
Error: model 'gpt-4' not found (status code: 404). Is Ollama running? Run 'ollama serve'


In [15]:
from src.utils.config import settings


generator = OllamaGenerator(settings.llm)

query_constructor = MultiQueryConstructor(generator)
answerer = QueryAnswerer(generator)
vector_store = ChromaStore(settings.vector_store)

rag = RAGPipeline(
    query_constructor=query_constructor,
    vector_store=vector_store,
    answerer=answerer
)

answer = rag.query("What are the main themes in the book?", top_k=5)
print(answer)

2026-01-29 22:43:12 | INFO     | src.ingestion.vector_store.stores:__init__:19 - creating or getting the collection
2026-01-29 22:43:12 | INFO     | src.ingestion.vector_store.stores:__init__:25 - getting the embedder
2026-01-29 22:43:12 | INFO     | src.ingestion.embedding.embedder:__init__:19 - Loading SentenceTransformer model: all-MiniLM-L6-v2


⚠️ Ollama not running. Please start it:
   Run in terminal: ollama serve


2026-01-29 22:43:20 | INFO     | src.ingestion.embedding.embedder:__init__:27 - Model loaded: 384d on cpu
2026-01-29 22:43:20 | INFO     | src.ingestion.vector_store.stores:query:47 - quering the results
2026-01-29 22:43:20 | INFO     | src.ingestion.vector_store.stores:query:73 - finished the quering


AttributeError: 'list' object has no attribute 'content'